In [85]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pprint
%matplotlib inline

In [86]:
img = cv2.imread("./images/test10.jpg", cv2.IMREAD_COLOR)

#plt.figure(figsize=(15,12))
#cv2.imshow("image", img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

print(img.shape)

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#plt.figure(figsize=(15,12))
#plt.imshow(img_gray)
#cv2.imshow("img_gray", img_gray)

#img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#plt.figure(figsize=(15,12))
#plt.imshow(img_gray)
#cv2.imshow("img_hsv", img_hsv)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

img_blur = cv2.GaussianBlur(img_gray, (9, 9), 0)
#cv2.imshow("img_blur", img_blur)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

#cv2.imshow("img_blur", img_blur)

## 적당한 threshold 값을 찾아서
#ret, img_th = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
#th2 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,15,2)
#ret1, img_th1 = cv2.threshold(img_gray, 150, 230, cv2.THRESH_BINARY_INV)
#ret2, img_th2 = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#ret3, img_th3 = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
ret4, img_th4 = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#img_thresh = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 12)
#cv2.imshow("img_th1", img_th1)
#cv2.imshow("img_th2", img_th2)
#cv2.imshow("img_th3", img_th3)
#cv2.imshow("img_th4", img_th4)

cv2.imshow("img_th4", img_th4)

# 대상은 흰색, 배경은 까만색이어야 컨투어 찾기 가능
contours, hierachy = cv2.findContours(img_th4.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#img_result = cv2.drawContours(img, contours, -1, (0,255,0), 3)
#cv2.imshow("img_result", img_result)

cv2.waitKey(0)
cv2.destroyAllWindows()

rects = [cv2.boundingRect(each) for each in contours]
rects

(460, 1178, 3)


[(817, 250, 84, 8),
 (594, 246, 63, 9),
 (741, 242, 60, 27),
 (195, 239, 58, 30),
 (1026, 236, 56, 33),
 (321, 236, 59, 35),
 (404, 232, 84, 26),
 (181, 225, 84, 8),
 (726, 203, 40, 34),
 (912, 188, 84, 67),
 (830, 188, 58, 44),
 (446, 188, 33, 41),
 (411, 188, 32, 41),
 (600, 187, 49, 44),
 (92, 187, 49, 57),
 (1016, 184, 49, 45),
 (314, 183, 42, 44),
 (722, 182, 47, 19),
 (195, 182, 57, 33),
 (1074, 180, 8, 53),
 (773, 180, 26, 68),
 (662, 180, 8, 91),
 (566, 180, 8, 91),
 (372, 180, 21, 53),
 (148, 180, 24, 91),
 (502, 179, 54, 71)]

In [87]:
tmp = [w*h for (x,y,w,h) in rects]
tmp.sort()
tmp

[424,
 567,
 672,
 672,
 728,
 728,
 893,
 1113,
 1312,
 1353,
 1360,
 1620,
 1740,
 1768,
 1848,
 1848,
 1881,
 2065,
 2156,
 2184,
 2184,
 2205,
 2552,
 2793,
 3834,
 5628]

In [88]:
# 제일 큰 아이 두개를 빼고, 나머지의 넓이를 찾을 수 있습니다. 그래서 넓이값으로 다시 한 번 더 걸러주는거죠...
#rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>15000)and(w*h<500000))]
rects = [(x,y,w,h) for (x,y,w,h) in rects]
rects

[(817, 250, 84, 8),
 (594, 246, 63, 9),
 (741, 242, 60, 27),
 (195, 239, 58, 30),
 (1026, 236, 56, 33),
 (321, 236, 59, 35),
 (404, 232, 84, 26),
 (181, 225, 84, 8),
 (726, 203, 40, 34),
 (912, 188, 84, 67),
 (830, 188, 58, 44),
 (446, 188, 33, 41),
 (411, 188, 32, 41),
 (600, 187, 49, 44),
 (92, 187, 49, 57),
 (1016, 184, 49, 45),
 (314, 183, 42, 44),
 (722, 182, 47, 19),
 (195, 182, 57, 33),
 (1074, 180, 8, 53),
 (773, 180, 26, 68),
 (662, 180, 8, 91),
 (566, 180, 8, 91),
 (372, 180, 21, 53),
 (148, 180, 24, 91),
 (502, 179, 54, 71)]

In [89]:
img_result = []
img_for_class = img.copy()

margin_pixel = 60
print(rects)
for rect in rects:
    #[y:y+h, x:x+w]
    img_result.append(
        img_for_class[rect[1]-margin_pixel : rect[1]+rect[3]+margin_pixel, 
                      rect[0]-margin_pixel : rect[0]+rect[2]+margin_pixel])
    
    # Draw the rectangles
    cv2.rectangle(img, (rect[0], rect[1]), 
                  (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5)

#plt.figure(figsize=(15,12))
#plt.imshow(img)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(817, 250, 84, 8), (594, 246, 63, 9), (741, 242, 60, 27), (195, 239, 58, 30), (1026, 236, 56, 33), (321, 236, 59, 35), (404, 232, 84, 26), (181, 225, 84, 8), (726, 203, 40, 34), (912, 188, 84, 67), (830, 188, 58, 44), (446, 188, 33, 41), (411, 188, 32, 41), (600, 187, 49, 44), (92, 187, 49, 57), (1016, 184, 49, 45), (314, 183, 42, 44), (722, 182, 47, 19), (195, 182, 57, 33), (1074, 180, 8, 53), (773, 180, 26, 68), (662, 180, 8, 91), (566, 180, 8, 91), (372, 180, 21, 53), (148, 180, 24, 91), (502, 179, 54, 71)]
